<a href="https://colab.research.google.com/github/iamVaibhav100/multilevel_dog_breed_classification/blob/main/dog_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End-to-End Multi-level Dog breed Classification

Building an end-to-end multi-level dog breed classifier using TensorFlow.

### 1. Problem

Given an image of a dog, identify the breed.

### 2. Data

Data used is from Kaggle's dog breed identification competition.

Link : https://www.kaggle.com/c/dog-breed-identification/data

### 3. Evaluation

Evaluation is a file with prediction probabilities for each dog breed of each test image.

https://www.kaggle.com/competitions/dog-breed-identification/overview/evaluation

### 4. Features

Information about the data used:
* Data is a collection of images (unstructured data) hence deep learning approach is used.
* There are 120 different breeds of dogs available in the data used.
* There are around 10,000+ images for training set (labelled data).
* There are around 10,000+ images for testing set (no label).


In [ ]:
#!unzip "drive/MyDrive/Dog_Vision/dog-breed-identification.zip" -d "drive/MyDrive/Dog_Vision"

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
print("version:", hub.__version__)
print("version:", tf.__version__)

# checking for GPU avalability
print("GPU", "available" if tf.config.list_physical_devices("GPU") else "not available")

version: 0.12.0
version: 2.9.2
GPU available
